# Data conversions

## Load data

In [ ]:
import numpy as np
import pandas as pd
import requests
import json

In [ ]:
# These are the segments from the GitHub repository, which should line up with the Jason Riggle chart
segments = requests.get('https://raw.githubusercontent.com/anna-hope/phonemes/master/phonemes.json').json()

segments = pd.json_normalize([
    {
        'symbol': key,
        'name': value['name'],
        **value['features']
    }
    for key, value in segments.items()
])
segments.set_index('symbol', inplace=True)
segments.head()

In [ ]:
# This is the raw data from Pheatures
pheatures = pd.read_csv('../src/assets/base-features.tsv', delimiter='\t')

del pheatures['Sound']
pheatures.rename(columns={'Unicode': 'symbol'}, inplace=True)
pheatures.set_index('symbol', inplace=True)
pheatures.head()

In [ ]:
pheatures.reset_index().replace('-', False).replace('+', True).replace('0', 0).to_json('../src/assets/base-features.json', orient='records', indent=2)

In [ ]:
pheatures.insert(0, 'name', [
    segments.loc[symbol]['name']
    if symbol in segments.name
    else ''
    for symbol in pheatures.index
])

In [ ]:
pheatures

In [ ]:
print('in pheatures but not chart: ', ', '.join(pheatures[~pheatures.index.isin(segments.index)].index))
print()
print('in chart but not pheatures: ', ', '.join(segments[~segments.index.isin(pheatures.index)].index))

In [ ]:
segments_cols = set([
    col            
        .replace('ant', 'anterior')
        .replace('son', 'sonorant')
        .replace('cons', 'consonantal')
        .replace('CG', 'constr gl')
        .replace('SG', 'spread gl')
        .replace('syl', 'syllabic')
        .replace('dist', 'distributed')
        .replace('cont', 'continuant')
        .replace('del_rel', 'delayed release')
    for col in segments.columns
])
pheatures_cols = set(pheatures.columns)
print('in pheatures but not chart: ', ', '.join(segments_cols.difference(pheatures_cols)))
print()
print('in chart but not pheatures: ', ', '.join(pheatures_cols.difference(segments_cols)))

In [ ]:
row = segments.iloc[0].sort_index()
row.drop(['ATR', 'name', 'pharyngeal'], inplace=True)

my_row = my_segments.loc[row.name].sort_index()
my_row.drop(['approximant', 'front', 'labiodental', 'long', 'stress', 'tap', 'trill'], inplace=True)